In [1]:
import geopandas as gpd
import rasterio as rio

In [2]:
# open and store rasters
compound_inun = rio.open('data_github/compound_inun.tif','r')
riv_inun = rio.open('data_github/030202_201809190900_depth_map_projUTM18n.tif','r')
surge_inun = rio.open('data_github/florence_surge_max_compr_lzw.tif','r')

In [3]:
# read points from shapefiles

# riverine high water marks
val_riv = gpd.read_file('data_github/shp/HWM.shp')
val_riv.index = range(len(val_riv))
pts_riv = val_riv.to_crs('epsg:26918').geometry
coords_utm_riv = [(pt.xy[0][0], pt.xy[1][0]) for pt in pts_riv]

# surge peak water level
val_surge = gpd.read_file('data_github/shp/Peak_water_level.shp')
val_surge.index = range(len(val_surge))
pts_surge = val_surge.to_crs('epsg:26918').geometry
coords_utm_surge = [(pt.xy[0][0], pt.xy[1][0]) for pt in pts_surge]

In [4]:
# Sample rasters at every point location and store values in geopandas dataframes

# riverine
val_riv['compound_inun value'] = [x[0] for x in compound_inun.sample(coords_utm_riv)]
val_riv['riv_inun value'] = [x[0] for x in riv_inun.sample(coords_utm_riv)]
val_riv['surge_inun value'] = [x[0] for x in surge_inun.sample(coords_utm_riv)]

# surge
val_surge['compound_inun value'] = [x[0] for x in compound_inun.sample(coords_utm_surge)]
val_surge['riv_inun value'] = [x[0] for x in riv_inun.sample(coords_utm_surge)]
val_surge['surge_inun value'] = [x[0] for x in surge_inun.sample(coords_utm_surge)]

In [5]:
val_riv.to_file('data_github/shp/HWM_joined.shp')
val_surge.to_file('data_github/shp/Peak_water_level_joined.shp')

C:\Software\Anaconda\Temp/ipykernel_393244/1824908753.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  val_riv.to_file('data_github/shp/HWM_joined.shp')
C:\Software\Anaconda\Temp/ipykernel_393244/1824908753.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  val_surge.to_file('data_github/shp/Peak_water_level_joined.shp')
